In [121]:
from utils import download

download('https://archive.ics.uci.edu/ml/machine-learning-databases/00591/name_gender_dataset.csv', 'Downloads/name_gender_dataset.csv')

[██████████████████████████████████████████████████]


In [122]:
import pandas as pd

name_gender_df = pd.read_csv('Downloads/name_gender_dataset.csv')

In [162]:
name_gender_df[name_gender_df['Name'] == 'Appelez']

,Name,Gender,Count,Probability


In [217]:
import os 
import json
import spacy
import itertools
import pprint
import requests
import urllib
from tqdm.notebook import tqdm

nlp = spacy.load('en_core_web_lg')

PATH = '../../data/separated_by_email/'

email_names_li = []

t = tqdm(os.listdir(PATH))

for email in t:
    t.set_description(email)
    
    if email.startswith('.'):
        continue
        
    j = json.load(open(os.path.join(PATH, email)))
    
    if 'X-TIKA:content' not in j:
        continue
    
    content = j['X-TIKA:content']
    doc = nlp(content)
    
    # check entity if is a person, if it is, split white spaces
    nameWords = [name.capitalize() for ee in doc.ents if ee.label_ == 'PERSON' for name in ee.text.split() if name.isalpha()]
    
    email_names_li += nameWords

  0%|          | 0/4032 [00:00<?, ?it/s]

In [296]:
import numpy as np
df = pd.DataFrame(email_names_li, columns=['Name'],dtype="string")
df['Count'] = pd.Series(np.ones(len(df)))
# df = df.groupby(['Name']).count().reset_index()
df_count = df.groupby(['Name']).agg({'Count': 'sum'}).sort_values(by=['Count'], ascending=False)
# df_c
df_absFreq = pd.DataFrame(df['Name'].value_counts(normalize=True) * 100)
df_absFreq.columns = ['Probability']

df = pd.merge(df_count, df_absFreq, left_index=True, right_index=True)
df['Count'] = df['Count'].astype(int)
df = df.reset_index().rename(columns={'index': 'Name'})
df.to_csv('Downloads/name_gender_dataset_emails.csv')
df

,Name,Count,Probability
0,Mrs,582,1.711463
1,Charles,391,1.149797
2,Abacha,380,1.11745
3,Taylor,350,1.02923
4,John,343,1.008646
...,...,...,...
4888,Fills,1,0.002941
4889,Fils,1,0.002941
4890,Permettrez,1,0.002941
4891,Permettre,1,0.002941


In [306]:
import os 
import json
import spacy
import itertools
import pprint
import requests
import urllib
from tqdm.notebook import tqdm

nlp = spacy.load('en_core_web_lg')

PATH = '../../data/separated_by_email/'
FIREBASE_URL = 'https://dsci-550-default-rtdb.firebaseio.com/'
FIREBASE_TASK5_URL = urllib.parse.urljoin(FIREBASE_URL, 'assignment-1/task-5-additional-features/')

t=tqdm(os.listdir(PATH))

for email in t:
    t.set_description(email)
    if email.startswith('.'):
        continue
    
#     print(email)
    j = json.load(open(os.path.join(PATH, email)))
    
    if 'X-TIKA:content' not in j:
        continue
    
    content = j['X-TIKA:content']
    doc = nlp(content)
    
    # check entity if is a person, if it is, split white spaces
    nameWords = [name for ee in doc.ents if ee.label_ == 'PERSON' for name in ee.text.split() if name.isalpha()]
    
    uploadJSON = {'name gender': {"name words": nameWords, "names": []}}
    
    for name in nameWords:
        nameCap = name.capitalize()
        lookup = name_gender_df[name_gender_df['Name'] == nameCap]
        lookup = lookup.sort_values(by=['Count'],ascending=False)
        
        lookup_emails = df[df['Name'] == nameCap]
        lookup_emails = lookup_emails.sort_values(by=['Count'],ascending=False)
        
        if not lookup.empty:
            uploadJSON['name gender']['names'].append(lookup.iloc[[0],:].to_dict(orient='records')[0])
            lookup_count = uploadJSON['name gender']['names'][-1]['Count']
            uploadJSON['name gender']['names'][-1]['Count'] = {'by_lookup': lookup_count, 'by_email': None}
            lookup_prob = uploadJSON['name gender']['names'][-1]['Probability']
            uploadJSON['name gender']['names'][-1]['Probability'] = {'by_lookup': lookup_prob, 'by_email': None}
        else:
            uploadJSON['name gender']['names'].append({'Name': name, 'Gender': None, 'Count': {'by_lookup':None, 'by_email':None}, 'Probability': {'by_lookup':None, 'by_email':None}})
        
        if not lookup_emails.empty:
            l = lookup_emails.iloc[[0],:].to_dict(orient='records')[0]
            lookup_count = l['Count']
            uploadJSON['name gender']['names'][-1]['Count']['by_email'] = lookup_count
            lookup_prob = l['Probability']
            uploadJSON['name gender']['names'][-1]['Probability']['by_email'] = lookup_prob
        else:
            uploadJSON['name gender']['names'].append({'Name': name, 'Gender': None, 'Count': {'by_lookup':None, 'by_email':None}, 'Probability': {'by_lookup':None, 'by_email':None}})
        
#     pp = pprint.PrettyPrinter(indent=1)
#     pp.pprint(uploadJSON)
    
    url = urllib.parse.urljoin(FIREBASE_TASK5_URL, email)
    requests.patch(url, json = uploadJSON)
    
#     # remove emails
#     email_pattern = '^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
#     emails = [re.findall(email_pattern, word, flags=0) for word in nameWords]
#     emails = list(itertools.chain.from_iterable(emails))
    
#     link_pattern =r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
#     links = [re.findall(link_pattern, word, flags=0) for word in nameWords]
#     links = list(itertools.chain.from_iterable(links))
    

  0%|          | 0/4032 [00:00<?, ?it/s]